In [1]:
import keras
from keras import layers
import numpy as np
import random
import io

In [2]:
with io.open('Konstityciya_Ykrajni.txt', encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  # We remove newlines chars for nicer display
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype="bool")
y = np.zeros((len(sentences), len(chars)), dtype="bool")
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Corpus length: 109450
Total chars: 74
Number of sequences: 36470


In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [4]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

epochs = 10
batch_size = 128

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("Generating text after epoch: %d" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("   Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('   Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("   Generated: ", generated)
        print("-")

285/285 [==============================] - 60s 195ms/step - loss: 2.3992

Generating text after epoch: 0
   Diversity: 0.2
   Generating with seed: "ується недоторканність житла.  не допуск"
   Generated:  она до стаття 111 ди прави приправа статорід    стаття 111 ди приди доводна достату дородо стаття 111 ди призадоном до статов до від 11. раді україни в державної стаття 111 ди приди дов. в державної дороди україни та судоном до доржавної достаття 111 див. в рішення достату державної радони дороди доряду дороди достату достату дорядо конституційного суду n 1) прида на дорядо стаття 111 ди призадент
-
   Diversity: 0.5
   Generating with seed: "ується недоторканність житла.  не допуск"
   Generated:  та україни порідачення статор дорадни україни у діяста конституції україни довста ду витрипрадентя конституції україни д правадо статової диження та приздачення о маження долади державної стаття 111 див. в дія довладання конституціїни україни.    офіці закраї реди україни.   стаття 111 ди.

In [10]:
from transformers import pipeline

generator = pipeline("text-generation", model="distilgpt2")
generator("Everybody want to",
    max_length=30)

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

C:\Users\Art\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Art\.cache\huggingface\hub\models--distilgpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/distilgpt2/e1ff18884359fe8beb795a5f414feb85a6ce3d929ad019c0d958c039d2b94a1b?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1705337193&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNTMzNzE5M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9kaXN0aWxncHQyL2UxZmYxODg4NDM1OWZlOGJlYjc5NWE1ZjQxNGZlYjg1YTZjZTNkOTI5YWQwMTljMGQ5NThjMDM5ZDJiOTRhMWI%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=QAIUrftO0ipKoo-ujt8sUIqDGuqy3uvEW3dSNeR-HVRLIlTJ0NTBAfNbvctgWodTVBNugcMCIEhcSTRbbDupCq2jzHot-yEqgD5i9TZZPlWwt2qMwWhq-G5P8C033lBVCcBnB29tG-byAvcn0q1XdnmZv8-K3CL8Cf4VIuBOKwV-WMjclpXpqLaNm3sR9QfvP3tEWDc4X6eTmdYp6zkjKuwOYr8sOTq9QincZDMlgrrjUTKoVFR4M8Vc4k4xnhZYSCh-pHwt2LbDkMXHwY1eiWUSaokafI3c0jEx9YWCtGPvnwxoTSUXAD-SYTLUCFh3u5pWhE9n2saa%7EDI6z%7E1pzw__&Key-Pair-Id=KVTP0A1DKRTAX: HTTPSConnectionPool(

model.safetensors:  24%|##3       | 83.9M/353M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Everybody want to do a great job at it, but it shouldn\'t be that way."'}]